<a href="https://colab.research.google.com/github/raju-prudhvi/BA-820-Group15/blob/main/BA820_G15_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas as pd
import numpy as np
import missingno as msno
import os
import re
import nltk
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from nltk.tokenize import sent_tokenize
from wordcloud import WordCloud
from nltk.sentiment import SentimentIntensityAnalyzer
import gensim.downloader as api
from sklearn.preprocessing import StandardScaler

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning, message="The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning")

# Coded by Himani

In [ ]:
od.download(
    "https://www.kaggle.com/datasets/deepshah16/song-lyrics-dataset")

#Coded by Himani

100%|██████████| 19.9M/19.9M [00:01<00:00, 19.2MB/s]


In [ ]:
directory = '/content/song-lyrics-dataset/csv/'
dfs = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        dfs.append(df)

df_concat = pd.concat(dfs, ignore_index=True)
df_concat

#Coded by Madhumitha

In [ ]:
df=df_concat
df.to_excel('df_concat.xlsx', index=False)

#Coded by Madhumitha

In [ ]:
df.info()

#Coded by Madhumitha

In [ ]:
df.head()
#Coded by Madhumitha

In [ ]:
df_cleaned=df.copy()
df_cleaned.drop('Unnamed: 0', axis=1, inplace=True)
df_cleaned

#Coded by Himani

,Artist,Title,Album,Year,Date,Lyric
0,Dua Lipa,New Rules,Dua Lipa,2017.0,2017-06-02,one one one one one talkin' in my sleep at n...
1,Dua Lipa,Don’t Start Now,Future Nostalgia,2019.0,2019-11-01,if you don't wanna see me did a full 80 craz...
2,Dua Lipa,IDGAF,Dua Lipa,2017.0,2017-06-02,you call me all friendly tellin' me how much y...
3,Dua Lipa,Blow Your Mind (Mwah),Dua Lipa,2016.0,2016-08-26,i know it's hot i know we've got something tha...
4,Dua Lipa,Be the One,Dua Lipa,2015.0,2015-10-30,i see the moon i see the moon i see the moon o...
...,...,...,...,...,...,...
6022,Post Malone,Lithium,NaN,2020.0,2020-04-24,i'm so happy 'cause today i found my friends t...
6023,Post Malone,Something in the Way,NaN,2020.0,2020-04-24,verse underneath the tarp has sprung a leak a...
6024,Post Malone,In Bloom,NaN,2020.0,2020-04-24,sell the kids for food weather changes moods s...
6025,Post Malone,Territorial Pissings,NaN,2020.0,2020-04-24,when i was an alien cultures weren't opinions ...


#1.1 Null Handling

In [ ]:
df_cleaned.isna().sum()

#Coded by Himani

Artist       0
Title        0
Album     1574
Year      1686
Date      1686
Lyric       46
dtype: int64

In [ ]:
plt.figure(figsize=(10, 8))
msno.matrix(df_cleaned, figsize=(10, 6), fontsize=12, sparkline=False, labels=True)

plt.title('Figure 1.1 Missing Values Matrix', fontsize=16)
plt.xticks(rotation=90, ha='right')
plt.yticks(fontsize=12)
plt.xlabel('Features', fontsize=14)
plt.ylabel('Entries', fontsize=14)
plt.tight_layout()
plt.show()

#Coded by Madhumitha

In [ ]:
unique_artist =df_cleaned['Artist'].unique()
unique_artist

num_unique = len(unique_artist)
print("Number of Unique Artists:",num_unique)

#Coded by Madhumitha

In [ ]:
df_null = df_cleaned[['Artist','Album','Year', 'Date', 'Lyric']]

tot_artist = df_cleaned['Artist'].value_counts()
null_year_percentage = (df_null[df_null['Year'].isna()].groupby('Artist').size() * 100 / tot_artist).rename('Artist_Year%')
null_date_percentage = (df_null[df_null['Date'].isna()].groupby('Artist').size() * 100 / tot_artist).rename('Artist_Date%')
null_album_percentage = (df_null[df_null['Album'].isna()].groupby('Artist').size() * 100 / tot_artist).rename('Artist_Album%')
null_lyric_percentage = (df_null[df_null['Lyric'].isna()].groupby('Artist').size() * 100 / tot_artist).rename('Artist_Lyric%')

df_null = pd.concat([null_year_percentage, null_date_percentage, null_album_percentage, null_lyric_percentage], axis=1)
df_null.reset_index(inplace=True)
df_null

#Coded by Himani

In [ ]:
plt.figure(figsize=(20, 6))

df_null_sort=df_null.sort_values(by='Artist_Lyric%', ascending = False).reset_index()

sns.barplot(x='index', y='Artist_Lyric%', data=df_null_sort)

plt.xlabel('Artist', fontsize=12)
plt.ylabel('%null', fontsize=12)
plt.title('Figure 1. 2 Null Values of Lyrics by Artist', fontsize=14)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

#Coded by Himani